In [1]:
from config import *
from data_processer import CSCDataset
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AdamW,
    BertForSequenceClassification,
    BertTokenizer,
    get_scheduler,
)

In [3]:
def cal_err(raw_sentence, pred_sentence, corr_sentence):
    matrices = ["over_corr", "total_err", "true_corr"]
    char_level = {key: 0 for key in matrices}
    sent_level = {key: 0 for key in matrices}

    f1 = f2 = 0
    for i, c in enumerate(raw_sentence):
        pc, cc = pred_sentence[i], corr_sentence[i]

        if cc != c:
            char_level["total_err"] += 1
            char_level["true_corr"] += pc == cc
            f1 = 1
        elif pc != cc:
            char_level["over_corr"] += 1
            f2 = 1

    if f1:
        sent_level["true_corr"] += all(pred_sentence == corr_sentence)
        sent_level["total_err"] += f1
    sent_level["over_corr"] += f2

    return char_level, sent_level

In [4]:
def test(model, tokenizer, test_data_loader):
    model.eval()
    total_loss = 0
    matrices = ["over_corr", "total_err", "true_corr"]
    test_char_level = {key: 0 for key in matrices}
    test_sent_level = {key: 0 for key in matrices}

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)

            loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()

            t = torch.argmax(outputs, dim=-1)
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)

            for i in range(len(t)):
                char_level, sent_level = cal_err(input_ids[i], nt[i], labels[i])
                test_char_level = {
                    key: test_char_level[key] + v for key, v in char_level.items()
                }
                test_sent_level = {
                    key: test_sent_level[key] + v for key, v in sent_level.items()
                }
        print(total_loss / len(test_data_loader), test_char_level, test_sent_level)

In [5]:
from config import *
from models import Seq2SeqModel
from torch.nn.functional import cross_entropy
from tqdm import tqdm

# temp_data
nhead = 4  # 多头注意力机制的头数
num_decoder_layers = 2
dim_feedforward = 3072
max_seq_len = 128
dropout = 0.1

if __name__ == "__main__":
    tokenizer = BertTokenizer.from_pretrained(checkpoint)
    bert = BertModel.from_pretrained(checkpoint)

    train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)
    train_data_loader = DataLoader(
        train_dataset, num_workers=0, shuffle=True, batch_size=16
    )

    test_dataset = CSCDataset(
        [SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer
    )
    test_data_loader = DataLoader(
        test_dataset, num_workers=0, shuffle=True, batch_size=32
    )

    model = Seq2SeqModel(
        bert, nhead, num_decoder_layers, dim_feedforward, max_seq_len, dropout
    )

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    epochs = 10
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(
            enumerate(train_data_loader),
            desc=f"Epoch:{epoch+1}/{epochs}",
            total=len(train_data_loader),
        )

        for i, batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

            # 反向传播在这，故labels不需要传入模型
            loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

        t = torch.argmax(outputs, dim=-1)
        nt = t * attention_mask
        pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
        # print(pred)
        # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

        for i, v in enumerate(nt):
            r, l = input_ids[i], labels[i]
            print(tokenizer.decode(r, skip_special_tokens=True))
            print(tokenizer.decode(v, skip_special_tokens=True))
            print(tokenizer.decode(l, skip_special_tokens=True))
            print(cal_err(r, v, l))

        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")
        test(model, tokenizer, test_data_loader)

preprocessing sighan dataset: 2339it [00:00, 933886.44it/s]
preprocessing sighan dataset: 100%|██████████| 2339/2339 [00:00<00:00, 1522184.18it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 890855.45it/s]
preprocessing sighan dataset: 100%|██████████| 3437/3437 [00:00<00:00, 1409309.11it/s]


共3437句，共170330字，最长的句子有258字


/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:1/10: 100%|██████████| 147/147 [00:09<00:00, 14.76it/s, loss=2.235]
/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


...

希 望 你 們 那 個 很 特 別 的 一 天 過 的 很 開 心 ！
希 望 你 們 那 個 很 特 別 的 一 天 過 的 很 開 心 ！
希 望 你 們 那 個 很 特 別 的 一 天 過 得 很 開 心 ！
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
他 等 一 下 ， 戶 然 他 看 到 一 輛 汽 車 。
他 等 一 下 ， 戶 然 他 看 到 一 輛 汽 車 。
他 等 一 下 ， 忽 然 他 看 到 一 輛 汽 車 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
起 點 五 十 分 公 車 來 了 ， 可 是 公 車 裡 很 多 人 ， 好 像 進 不 去 。
起 點 五 十 分 公 車 來 了 ， 可 是 公 車 裡 很 多 人 ， 好 像 進 不 去 。
七 點 五 十 分 公 車 來 了 ， 可 是 公 車 裡 很 多 人 ， 好 像 進 不 去 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 8 Loss: 0.37050490942942044
nan {'over_corr': 5628, 'total_err': 5278, 'true_corr': tensor(341, device='cuda:0')} {'over_corr': 2014, 'total_err': 3436, 'true_corr': 66}


Epoch:9/10: 100%|██████████| 147/147 [00:09<00:00, 15.94it/s, loss=0.231]


請 想 像 一 位 未 來 總 統 候 選 人 ； 如 果 媒 體 捉 去 她 小 時 候 影 片 ， 發 現 甚 麼 缺 點 ， 就 算 是 小 時 候 ， 選 民 必 然 會 受 不 好 的 影 像 。
請 想 像 一 位 未 來 總 統 候 選 人 ； 如 果 媒 體 好 去 她 小 時 候 影 片 ， 發 現 甚 麼 缺 點 ， 就 算 是 小 時 候 ， 選 民 必 然 會 受 不 好 的 影 像 。
請 想 像 一 位 未 來 總 統 候 選 人 ； 如 果 媒 體 捉 去 她 小 時 候 影 片 ， 發 現 甚 麼 缺 點 ， 就 算 是 小 時 候 ， 選 民 必 然 會 受 不 好 的 影 響 。
({'over_corr': 1, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
裝 錄 音 機 在 每 個 教 室 裡 有 很 多 的 好 處 ， 對 我 來 說 莊 錄 影 機 在 教 室 裡 的 好 處 比 壞 處 多 了 。
裝 錄 音 機 在 每 個 教 室 裡 有 很 多 的 好 處 ， 對 我 來 說 裝 錄 影 機 在 教 室 裡 的 好 處 比 壞 處 多 了 。
裝 錄 影 機 在 每 個 教 室 裡 有 很 多 的 好 處 ， 對 我 來 說 裝 錄 影 機 在 教 室 裡 的 好 處 比 壞 處 多 了 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
圖 書 館 沒 有 門 票 也 不 用 附 那 邊 的 電 費 ， 有 那 麼 多 空 的 話 就 去 那 邊 看 書 可 以 得 到 很 多 很 多 知 識 。
圖 書 館 沒 有 們 漂 也 不 用 附 那 邊 的 電 費 ， 有 那 麼 多 空 的 話 就 去 那 邊 看 書 可 以 得 到 很 多 很 多 知 識 。
圖 書 館 沒 有 門 票 也 不 用 付 那 邊 的 電 費 ， 有 那 麼 多 空 

Epoch:10/10: 100%|██████████| 147/147 [00:09<00:00, 15.95it/s, loss=0.243]


一 方 面 可 以 不 斷 的 訓 練 自 己 的 人 生 知 識 ， 也 可 以 證 明 自 己 ， 提 高 女 人 在 社 會 的 地 位 。
一 方 面 可 以 不 斷 的 訓 練 自 己 的 人 生 知 識 ， 也 可 以 證 明 自 己 ， 提 高 女 人 在 社 會 的 地 位 。
一 方 面 可 以 不 斷 地 訓 練 自 己 的 人 生 知 識 ， 也 可 以 證 明 自 己 ， 提 高 女 人 在 社 會 的 地 位 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
聽 說 你 生 病 了 ， 也 不 看 到 你 上 個 。
聽 說 你 生 病 了 ， 也 不 看 到 你 上 個 。
聽 說 你 生 病 了 ， 也 不 看 到 你 上 課 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
還 有 隨 便 丟 掉 的 懷 人 抓 他 。 被 他 罰 很 多 錢 才 對 的 。
還 有 隨 便 丟 掉 的 懷 人 抓 他 。 被 他 罰 很 多 錢 才 對 的 。
還 有 隨 便 丟 掉 的 壞 人 抓 他 。 被 他 罰 很 多 錢 才 對 的 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 10 Loss: 0.30734801474882634
nan {'over_corr': 5085, 'total_err': 5278, 'true_corr': tensor(349, device='cuda:0')} {'over_corr': 1896, 'total_err': 3436, 'true_corr': 70}


In [7]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(
        enumerate(train_data_loader),
        desc=f"Epoch:{epoch+1}/{epochs}",
        total=len(train_data_loader),
    )

    for i, batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device).type(torch.float)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, src_mask=attention_mask)
        logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

        # 反向传播在这，故labels不需要传入模型
        loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

    t = torch.argmax(outputs, dim=-1)
    nt = t * attention_mask
    pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
    # print(pred)
    # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

    for i, v in enumerate(nt):
        r, l = input_ids[i], labels[i]
        print(tokenizer.decode(r, skip_special_tokens=True))
        print(tokenizer.decode(v, skip_special_tokens=True))
        print(tokenizer.decode(l, skip_special_tokens=True))
        print(cal_err(r, v, l))

    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")
    test(model, tokenizer, test_data_loader)

Epoch:1/10: 100%|██████████| 147/147 [00:09<00:00, 15.90it/s, loss=0.169]


上 課 時 間 十 七 點 四 十 分 ， 八 點 我 才 上 車 去 我 遲 到 了 ， 怎 麼 辦 ？
上 課 時 間 十 七 點 四 十 分 ， 八 點 我 才 上 車 去 我 遲 到 了 ， 怎 麼 辦 ？
上 課 時 間 是 七 點 四 十 分 ， 八 點 我 才 上 車 去 我 遲 到 了 ， 怎 麼 辦 ？
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
謝 謝 看 我 的 意 見 ， 而 且 請 你 考 慮 我 的 意 見 以 下 。
謝 謝 看 我 的 意 見 ， 而 且 請 你 考 慮 我 的 意 見 以 下 。
謝 謝 看 我 的 意 見 ， 而 且 請 你 考 慮 我 的 意 見 一 下 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
由 於 昨 天 家 裡 比 較 熱 ， 所 以 我 們 打 開 窗 戶 ， 怎 麼 可 能 予 想 ， 外 面 臭 得 要 命 。
由 於 昨 天 家 裡 比 較 熱 ， 所 以 我 們 打 開 窗 戶 ， 怎 麼 可 能 予 想 ， 外 面 臭 得 要 命 。
由 於 昨 天 家 裡 比 較 熱 ， 所 以 我 們 打 開 窗 戶 ， 怎 麼 可 能 預 想 ， 外 面 臭 得 要 命 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 1 Loss: 0.2833105677447351
nan {'over_corr': 5342, 'total_err': 5278, 'true_corr': tensor(425, device='cuda:0')} {'over_corr': 197

Epoch:2/10: 100%|██████████| 147/147 [00:09<00:00, 15.88it/s, loss=0.240]


對 這 些 人 要 看 綠 色 至 少 １５ 分 鐘 、 綠 色 會 給 人 的 研 舒 服 的 感 覺 、 但 是 沒 有 真 正 綠 色 的 環 境 、 那 我 們 又 要 開 始 研 究 為 了 綠 色 的 綠 色 、 那 也 要 花 錢 。
對 這 些 人 要 看 綠 色 至 少 １５ 分 鐘 、 綠 色 會 給 人 的 研 舒 服 的 感 覺 、 但 是 沒 有 真 正 綠 色 的 環 境 、 那 我 們 又 要 開 始 研 究 為 了 綠 色 的 綠 色 、 那 也 要 花 錢 。
對 這 些 人 要 看 綠 色 至 少 １５ 分 鐘 、 綠 色 會 給 人 的 顏 舒 服 的 感 覺 、 但 是 沒 有 真 正 綠 色 的 環 境 、 那 我 們 又 要 開 始 研 究 為 了 綠 色 的 綠 色 、 那 也 要 花 錢 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
我 們 也 有 證 明 的 質 料 給 您 思 考 ， 也 得 看 您 願 不 願 意 跟 我 們 座 談 。
我 們 也 有 證 明 的 質 料 給 您 思 考 ， 也 得 看 您 願 不 願 意 跟 我 們 坐 談 。
我 們 也 有 證 明 的 資 料 給 您 思 考 ， 也 得 看 您 願 不 願 意 跟 我 們 座 談 。
({'over_corr': 1, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
從 沒 有 噪 音 和 臭 味 ， 為 何 你 們 的 工 廠 最 近 會 造 出 那 麼 強 列 噪 音 與 臭 味 呢 ？
從 沒 有 噪 音 和 臭 味 ， 為 何 你 們 的 工 廠 最 近 會 造 出 那 麼 強 列 噪 音 與 臭 味 呢 ？
從 沒 有 噪 音 和 臭 味 ， 為 何 你 們 的 工 廠 最 近 會 造 出 那 麼 強 烈 噪 音 與 臭 味 呢 ？
({'over_corr'

Epoch:3/10: 100%|██████████| 147/147 [00:09<00:00, 15.85it/s, loss=0.238]


...

因 為 現 在 ， 我 們 再 得 這 一 時 代 ， 就 是 不 能 相 信 別 人 家 的 很 冷 淡 的 時 代 嘛 ！
因 為 現 在 ， 我 們 在 得 這 一 時 代 ， 就 是 不 能 相 信 別 人 家 的 很 冷 淡 的 時 代 嘛 ！
因 為 現 在 ， 我 們 在 的 這 一 時 代 ， 就 是 不 能 相 信 別 人 家 的 很 冷 淡 的 時 代 嘛 ！
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
現 在 網 路 已 經 可 以 無 限 上 網 以 及 數 度 變 很 快 ， 網 路 已 經 變 成 無 所 不 在 了 。
現 在 網 路 已 經 可 以 無 限 上 網 以 及 數 度 變 很 快 ， 網 路 已 經 變 成 無 所 不 在 了 。
現 在 網 路 已 經 可 以 無 限 上 網 以 及 速 度 變 很 快 ， 網 路 已 經 變 成 無 所 不 在 了 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
我 覺 得 裝 這 種 東 西 的 予 算 是 浪 費 ， 這 位 校 長 可 能 只 想 賺 錢 。
我 覺 得 裝 這 種 東 西 的 予 算 是 浪 費 ， 這 位 校 長 可 能 只 想 賺 錢 。
我 覺 得 裝 這 種 東 西 的 預 算 是 浪 費 ， 這 位 校 長 可 能 只 想 賺 錢 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 8 Loss: 0.16697549779398912
nan {'over_corr': 20318, 'total_err': 5278, 'tru

Epoch:9/10: 100%|██████████| 147/147 [00:09<00:00, 15.92it/s, loss=0.246]


我 知 道 每 個 做 父 母 的 人 都 很 關 心 他 們 的 孩 子 ， 然 而 個 遊 客 個 觀 念 ， 每 個 人 都 不 一 樣 。
我 知 道 每 個 做 父 母 的 人 都 很 關 心 他 們 的 孩 子 ， 然 而 個 遊 客 個 觀 念 ， 每 個 人 都 不 一 樣 。
我 知 道 每 個 做 父 母 的 人 都 很 關 心 他 們 的 孩 子 ， 然 而 各 有 各 的 觀 念 ， 每 個 人 都 不 一 樣 。
({'over_corr': 0, 'total_err': 4, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
首 先 請 客 位 讀 者 自 己 想 像 自 己 為 這 所 小 學 的 學 生 。
首 先 請 客 位 讀 者 自 己 想 像 自 己 為 這 所 小 學 的 學 生 。
首 先 請 各 位 讀 者 自 己 想 像 自 己 為 這 所 小 學 的 學 生 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
因 為 像 人 ， 牠 比 較 小 的 時 候 不 會 記 得 出 了 什 麼 事 ， 所 以 不 會 受 到 心 理 上 的 通 苦 。
因 為 像 人 ， 牠 比 較 小 的 時 候 不 會 記 得 出 了 什 麼 事 ， 所 以 不 會 受 到 心 裡 上 的 通 苦 。
因 為 像 人 ， 牠 比 較 小 的 時 候 不 會 記 得 出 了 什 麼 事 ， 所 以 不 會 受 到 心 理 上 的 痛 苦 。
({'over_corr': 1, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
Epoch 9 Loss: 0.15459681601345945
nan {'over_corr': 25670,

Epoch:10/10: 100%|██████████| 147/147 [00:09<00:00, 15.88it/s, loss=0.140]


「 學 校 黑 網 站 」 不 但 被 欺 負 的 人 的 私 人 隱 私 洩 漏 到 全 校 而 且 洩 漏 到 全 國 ， 甚 至 些 漏 到 全 球 。
「 學 校 黑 網 站 」 不 但 被 欺 負 的 人 的 私 人 隱 私 洩 漏 到 全 校 而 且 洩 漏 到 全 國 ， 甚 至 些 漏 到 全 球 。
「 學 校 黑 網 站 」 不 但 被 欺 負 的 人 的 私 人 隱 私 洩 漏 到 全 校 而 且 洩 漏 到 全 國 ， 甚 至 洩 漏 到 全 球 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
總 算 是 你 的 公 司 就 交 給 我 ， 可 是 我 要 有 時 間 已 翻 翻 看 我 自 己 的 工 作 ， 以 方 面 練 習 當 老 闆 人 的 方 法 。
總 算 是 你 的 公 司 就 交 給 我 ， 可 是 我 要 有 時 間 已 翻 翻 看 我 自 己 的 工 作 ， 以 方 面 練 習 當 老 闆 人 的 方 法 。
總 算 是 你 的 公 司 就 交 給 我 ， 可 是 我 要 有 時 間 以 翻 翻 看 我 自 己 的 工 作 ， 一 方 面 練 習 當 老 闆 人 的 方 法 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
即 然 我 要 跟 你 見 面 ， 可 是 應 該 幫 他 ， 可 不 可 以 改 天 嗎 ？
既 然 我 要 跟 你 見 面 ， 可 是 應 該 幫 他 ， 可 不 可 以 改 天 嗎 ？
既 然 我 要 跟 你 見 面 ， 可 是 應 該 幫 他 ， 可 不 可 以 改 天 嗎 ？
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 

In [6]:
class Trainer:
    def __init__(self, model):
        self.model = model
        self.matrices = ["over_corr", "total_err", "true_corr"]

    def train(self, dataloader, epoch):
        self.iteration(dataloader, epoch)

    def test(self, dataloader):
        self.iteration(dataloader, train=False)

    def iteration(self, dataloader, epochs=1, train=True):
        mode = "train" if train else "dev"
        model.train() if train else model.eval()

        for epoch in range(epochs):
            # matrices
            total_loss = 0
            char_level = {key: 0 for key in self.matrices}
            sent_level = {key: 0 for key in self.matrices}

            progress_bar = tqdm(
                dataloader,
                desc=f"{mode} Epoch:{epoch+1}/{epochs}",
                total=len(dataloader),
            )
            for batch in progress_bar:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, src_mask=attention_mask, tgt=labels)
                logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

                loss = cross_entropy(
                    logits, labels, ignore_index=tokenizer.pad_token_id
                )

                total_loss += loss.item()
                progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

    def cal_err(raw_sentence, pred_sentence, corr_sentence):
        char_level = {key: 0 for key in self.matrices}
        sent_level = {key: 0 for key in self.matrices}

        for i, c in enumerate(raw_sentence):
            pc, cc = pred_sentence[i], corr_sentence[i]
            f1 = f2 = False

            if cc != c:
                char_level["total_err"] += 1
                char_level["true_corr"] += pc == cc
                f1 = True
            elif pc != cc:
                char_level["over_corr"] += 1
                f2 = True

        # true_corr 未计算
        sent_level["total_err"] += f1
        sent_level["over_corr"] += f2

        return char_level, sent_level